In [1]:
library(tidyverse)
library(haven)
library(ivreg)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
census_data <- read_dta('cen_ind_2021_pumf_v2 2.dta')

In [3]:
cleaned_data <- census_data |> select(agegrp, yrim, lfact, marsth, hdgree, jobperm,CFInc_AT, PR1, NOC21, Gender, immstat) |> 
    filter(agegrp != 88) |> 
    filter(yrim != 8888 & yrim != 9999) |> 
mutate(yrim = case_when(
yrim ==1~ 1950,
yrim ==2 ~ 1957,
yrim ==3~1962,
yrim == 4 ~ 1967,
yrim ==5 ~ 1972,
yrim ==6 ~ 1977,
yrim ==7 ~ 1982,
yrim ==8 ~ 1987,
yrim ==9 ~ 1992,
yrim >= 1995 & yrim <= 2021 ~ yrim, TRUE ~ NA_real_))|>
    filter(lfact != 88 & lfact != 99) |>
    filter(marsth != 8) |>
    filter(hdgree != 88 & hdgree != 99) |>
    filter(jobperm != 8 & jobperm != 9) |>
    filter(CFInc_AT != 88) |>
    filter(PR1 != 88 & PR1 != 99) |>
    filter(NOC21 != 88 & NOC21 != 99)


In [4]:

statistic<- cleaned_data |> summarise(
    across(everything(),
      list(
        mean = ~ mean(.x, na.rm = TRUE),
        sd   = ~ sd(.x, na.rm = TRUE),
        min  = ~ min(.x, na.rm = TRUE),
        max  = ~ max(.x, na.rm = TRUE)
      )
    )
  )

statistic_tidy <- statistic |> t()

statistic_tidy

agegrp_mean,12.5840709
agegrp_sd,2.7017528
agegrp_min,6.0000000
agegrp_max,21.0000000
yrim_mean,2000.6679073
yrim_sd,14.5030381
yrim_min,1950.0000000
yrim_max,2020.0000000
lfact_mean,2.2227089
lfact_sd,3.0558378
lfact_min,1.0000000


In [5]:
census_subset <- cleaned_data %>% filter(agegrp == 16 | agegrp == 17 ) |>
    mutate(retired = ifelse(lfact %in% c(11,12,13), 1, 0)) |>
    mutate(female = ifelse(Gender == 1, 1, 0))
head(census_subset)

agegrp,yrim,lfact,marsth,hdgree,jobperm,CFInc_AT,PR1,NOC21,Gender,immstat,retired,female
<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>
16,1977,1,5,1,2,12,35,26,2,2,0,0
16,2006,1,2,9,2,27,35,2,2,2,0,0
17,1992,5,1,9,2,13,35,6,1,2,0,1
16,1995,1,1,7,1,11,48,12,1,2,0,1
17,1987,1,2,2,1,29,24,18,2,2,0,0
16,1982,1,2,4,1,29,48,21,2,2,0,0


In [6]:
census_subset <- census_subset %>% 
mutate(eligible = ifelse(agegrp == 17 & (immstat == 1 | (immstat %in% c(2,3) & (2021 - yrim) >= 10)), 1, 0))

head(census_subset)

agegrp,yrim,lfact,marsth,hdgree,jobperm,CFInc_AT,PR1,NOC21,Gender,immstat,retired,female,eligible
<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl>
16,1977,1,5,1,2,12,35,26,2,2,0,0,0
16,2006,1,2,9,2,27,35,2,2,2,0,0,0
17,1992,5,1,9,2,13,35,6,1,2,0,1,1
16,1995,1,1,7,1,11,48,12,1,2,0,1,0
17,1987,1,2,2,1,29,24,18,2,2,0,0,1
16,1982,1,2,4,1,29,48,21,2,2,0,0,0


In [7]:
model1 <- lm(retired ~ eligible, data = census_subset) 
summary(model1)


Call:
lm(formula = retired ~ eligible, data = census_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.2104 -0.2104 -0.1011 -0.1011  0.8989 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.101074   0.003259   31.01   <2e-16 ***
eligible    0.109334   0.005675   19.27   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3401 on 16252 degrees of freedom
Multiple R-squared:  0.02233,	Adjusted R-squared:  0.02227 
F-statistic: 371.2 on 1 and 16252 DF,  p-value: < 2.2e-16


In [8]:
model2 <- lm(retired ~ eligible + female + factor(marsth), data = census_subset) 
summary(model2)


Call:
lm(formula = retired ~ eligible + female + factor(marsth), data = census_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.2442 -0.1889 -0.1186 -0.0820  0.9439 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.088097   0.012025   7.326 2.48e-13 ***
eligible         0.111266   0.005676  19.602  < 2e-16 ***
female           0.040924   0.005492   7.451 9.71e-14 ***
factor(marsth)2 -0.006099   0.011885  -0.513   0.6079    
factor(marsth)3  0.003235   0.016666   0.194   0.8461    
factor(marsth)4 -0.031961   0.018403  -1.737   0.0824 .  
factor(marsth)5 -0.010454   0.014249  -0.734   0.4632    
factor(marsth)6  0.003885   0.017454   0.223   0.8239    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3396 on 16246 degrees of freedom
Multiple R-squared:  0.02611,	Adjusted R-squared:  0.02569 
F-statistic: 62.23 on 7 and 16246 DF,  p-value: < 2.2e-16


In [9]:
model3 <- lm(retired ~ eligible + female + factor(marsth) + factor(hdgree) + factor(jobperm), data = census_subset) 
summary(model3)


Call:
lm(formula = retired ~ eligible + female + factor(marsth) + factor(hdgree) + 
    factor(jobperm), data = census_subset)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.39931 -0.17910 -0.10305 -0.05218  1.02278 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)       0.148196   0.014466  10.244  < 2e-16 ***
eligible          0.108512   0.005636  19.255  < 2e-16 ***
female            0.041558   0.005508   7.545 4.77e-14 ***
factor(marsth)2  -0.001344   0.011767  -0.114 0.909098    
factor(marsth)3   0.003232   0.016492   0.196 0.844653    
factor(marsth)4  -0.033907   0.018209  -1.862 0.062607 .  
factor(marsth)5  -0.010651   0.014101  -0.755 0.450057    
factor(marsth)6  -0.001489   0.017276  -0.086 0.931293    
factor(hdgree)2  -0.032252   0.009085  -3.550 0.000386 ***
factor(hdgree)3  -0.048999   0.014558  -3.366 0.000765 ***
factor(hdgree)4  -0.001952   0.015924  -0.123 0.902419    
factor(hdgree)5  -0.045412   0.016500  -2.752

In [10]:
model4 <- lm(retired ~ eligible + female + factor(marsth) + factor(hdgree) + factor(jobperm) + CFInc_AT	 + factor(PR1) + factor(NOC21), data = census_subset) 
summary(model4)


Call:
lm(formula = retired ~ eligible + female + factor(marsth) + factor(hdgree) + 
    factor(jobperm) + CFInc_AT + factor(PR1) + factor(NOC21), 
    data = census_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.4670 -0.1714 -0.1024 -0.0411  1.0391 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       0.3299741  0.0768642   4.293 1.77e-05 ***
eligible          0.1090964  0.0056242  19.398  < 2e-16 ***
female            0.0392866  0.0063234   6.213 5.33e-10 ***
factor(marsth)2   0.0416092  0.0123302   3.375 0.000741 ***
factor(marsth)3   0.0480932  0.0168635   2.852 0.004351 ** 
factor(marsth)4  -0.0289757  0.0181543  -1.596 0.110490    
factor(marsth)5  -0.0060493  0.0140637  -0.430 0.667103    
factor(marsth)6   0.0073167  0.0172448   0.424 0.671362    
factor(hdgree)2  -0.0236251  0.0091966  -2.569 0.010211 *  
factor(hdgree)3  -0.0389440  0.0146893  -2.651 0.008029 ** 
factor(hdgree)4   0.0061813  0.0162472   0.380 0.703615  

In [11]:
census_subset <- census_subset |> mutate(yob = 2021 - agegrp)

In [12]:

iv_model <- ivreg(
  retired ~ eligible
          + female
          + factor(marsth)
          + factor(hdgree)
          + factor(jobperm)
          + CFInc_AT
          + factor(PR1)
          + factor(NOC21)
  | yob
  + female
  + factor(marsth)
  + factor(hdgree)
  + factor(jobperm)
  + CFInc_AT
  + factor(PR1)
  + factor(NOC21),
  data = census_subset
)

In [13]:
iv_model


Call:
ivreg(formula = retired ~ eligible + female + factor(marsth) +     factor(hdgree) + factor(jobperm) + CFInc_AT + factor(PR1) +     factor(NOC21) | yob + female + factor(marsth) + factor(hdgree) +     factor(jobperm) + CFInc_AT + factor(PR1) + factor(NOC21),     data = census_subset)

Coefficients:
     (Intercept)          eligible            female   factor(marsth)2  
        0.326913          0.115219          0.039668          0.041538  
 factor(marsth)3   factor(marsth)4   factor(marsth)5   factor(marsth)6  
        0.048128         -0.028903         -0.006408          0.006764  
 factor(hdgree)2   factor(hdgree)3   factor(hdgree)4   factor(hdgree)5  
       -0.023822         -0.039223          0.006063         -0.032616  
 factor(hdgree)6   factor(hdgree)7   factor(hdgree)8   factor(hdgree)9  
       -0.048969         -0.029673         -0.055527         -0.047566  
factor(hdgree)10  factor(hdgree)11  factor(hdgree)12  factor(hdgree)13  
       -0.038853         -0.069781   